# Композиции алгоритмов - практика

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("flights_sample.csv")
data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,11,12,4,OO,4622,N806SK,MSP,BOI,1132,...,1333.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,6,18,4,AA,2423,N4XPAA,MSP,DFW,1520,...,1952.0,1,0,0,NaN,0.0,0.0,0.0,125.0,0.0
2,2015,3,5,4,OO,5627,N114SY,CLT,ORD,1704,...,1844.0,1,0,0,NaN,0.0,0.0,6.0,24.0,0.0
3,2015,4,4,6,US,854,N206UW,ORD,PHL,1835,...,2116.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,9,23,3,UA,1082,N13248,ORD,EWR,1030,...,1318.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Feature Engineering Express

1) Оставьте колонки "MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"
                 
2) Удалите пропуски в датасете

3) Создайте столбец  target с 1 и 0: 1, если опоздание более чем на 10 минут, 0 - меньше или ровно 10 минут

4) Для листа cols закодируйте колонки с категориальными переменными в числа (т.е. текстовым переменным поставим в соответствие числа

5) Разбейте данные на тренировочную и тестовую выборку в соотношении 0.75, 0.25

In [4]:
data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]


data.dropna(inplace=True)


cols = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","ARRIVAL_DELAY"]


for item in cols:
    data[item] = data[item].astype("category").cat.codes + 1

X_train, X_test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1),
                                                    data["ARRIVAL_DELAY"],
                                                    test_size=0.25)

In [5]:
data.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DESTINATION_AIRPORT,ORIGIN_AIRPORT,AIR_TIME,DEPARTURE_TIME,DISTANCE,ARRIVAL_DELAY
0,11,12,4,10,4610,340,521,163.0,1131.0,1142,1
1,6,18,4,1,2420,388,521,109.0,1729.0,852,2
2,3,5,4,10,5614,530,371,100.0,1743.0,599,2
3,4,4,6,12,852,540,533,82.0,1829.0,678,1
4,9,23,3,11,1079,408,533,97.0,1023.0,719,1


## Поддерживающее задание 

Создайте пользовательскую функцию print_auc с тремя параметрами:
- model: модель, которая будет делать прогнозы
- train: тренировочные данные
- test: тестовые данные

Функция возвращает два значения: roc_auc для тренировочной выборки и roc_auc для тестовой выборки

In [6]:
def print_auc(m, train, test): 
    return (metrics.roc_auc_score(y_train, m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

Все бустинговые библиотеки поддерживают формат обучения моделей, как в sklearn, так и свои собственные элементы

# Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# угадайте, какой код тут
model_rf = RandomForestClassifier()


model_rf.fit(X_train, y_train)

In [ ]:
y_pred_rf = model_rf.predict(X_test)

In [ ]:
y_pred_rf

In [ ]:
y_pred_rf_proba = model_rf.predict_proba(X_test)[:, 1]
print(y_pred_rf_proba)

In [ ]:
print(roc_auc_score(y_test, y_pred_rf_proba))

### Попробуем найти, с каким количеством деревьев лучше

In [ ]:
params = {'n_estimators': [10, 50],
          'max_features': [3, 5]}

model = RandomForestClassifier()

grid_cv = GridSearchCV(model, params, cv=3, scoring='roc_auc')

In [ ]:
grid_cv.fit(X_train, y_train)

In [ ]:
grid_cv.best_params_

In [ ]:
grid_cv.best_score_

In [ ]:
model_cv = grid_cv.best_estimator_

In [ ]:
y_pred_cv = model_cv.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_pred_cv))

# LightGBM

* [Документация LightGBM](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)

### Основные параметры

(lightgbm/catboost)

* objective – функционал, на который будет настраиваться композиция
* eta / learning_rate – темп (скорость) обучения
* num_iterations / n_estimators  – число итераций бустинга

#### Параметры, отвечающие за сложность деревьев
* max_depth – максимальная глубина 
* max_leaves / num_leaves – максимальное число вершин в дереве
* gamma / min_gain_to_split – порог на уменьшение функции ошибки при расщеплении в дереве
* min_data_in_leaf – минимальное число объектов в листе
* min_sum_hessian_in_leaf – минимальная сумма весов объектов в листе, минимальное число объектов, при котором делается расщепление 
* lambda – коэффициент регуляризации (L2)
* subsample / bagging_fraction – какую часть объектов обучения использовать для построения одного дерева 
* colsample_bytree / feature_fraction – какую часть признаков использовать для построения одного дерева 

Подбор всех этих параметров — настоящее искусство. Но начать их настройку можно с самых главных параметров: learning_rate и n_estimators. Обычно один из них фиксируют, а оставшийся из этих двух параметров подбирают (например, фиксируют n_estimators=1000 и подбирают learning_rate). Следующим по важности является max_depth. В силу того, что мы заинтересованы в неглубоких деревьях, обычно его перебирают из диапазона [3; 7].




### Вариант 0

In [7]:
from lightgbm import LGBMClassifier

In [8]:
model = LGBMClassifier()
model.fit(X_train, y_train)

y_pred_lgb = model.predict_proba(X_test)
print(roc_auc_score(y_test, y_pred_lgb[:, 1]))

0.7294852807023812


### Вариант 1

In [10]:
params_gbm = {
    "objective": 'binary',
    "max_depth": 5,
    "min_child_weight" : 2,
    "n_estimators": 100,
    "learning_rate": 0.1}

In [12]:
import lightgbm as lgb

In [34]:
d_train = lgb.Dataset(X_train, label=y_train)
model = lgb.train(params_gbm, d_train)
y_preg_lgb = model.predict(X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 428667, number of negative: 0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002420 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1527
[LightGBM] [Info] Number of data points in the train set: 428667, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Info] Start training from score 34.539576
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Warning] Stopped training because there are 

In [36]:
print(roc_auc_score(y_test, y_preg_lgb))

0.5


In [27]:
model.save_model('lightgbm.txt')

In [28]:
data['ARRIVAL_DELAY'].value_counts()

1    445789
2    125768
Name: ARRIVAL_DELAY, dtype: int64

In [29]:
y_train.value_counts()

1    334444
2     94223
Name: ARRIVAL_DELAY, dtype: int64

In [30]:
y_test.value_counts()

1    111345
2     31545
Name: ARRIVAL_DELAY, dtype: int64

## Образец поиска по решетке для LGB

In [37]:
# Parameter Tuning
model = lgb.LGBMClassifier()
param_dict = {"max_depth": [5, 10, 15],
              "learning_rate" : [0.1, 0.15, 0.3]}


grid_search = GridSearchCV(model, param_grid=param_dict, cv=3, n_jobs=-1)


grid_search.fit(X_train, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.15, 0.3],
                         'max_depth': [5, 10, 15]})

In [38]:
max_depth_best = grid_search.best_estimator_.max_depth
learning_rate_best = grid_search.best_estimator_.learning_rate
print('max_depth_best:', max_depth_best)
print('learning_rate_best:', learning_rate_best)
print('best_score_', grid_search.best_score_)

max_depth_best: 15
learning_rate_best: 0.3
best_score_ 0.8011066865422345


In [41]:
# Parameter Tuning
model_best = lgb.LGBMClassifier(max_depth=15, learning_rate=0.3)
model_best.fit(X_train, y_train)
y_pred_best = model_best.predict_proba(X_test)
print(roc_auc_score(y_test, y_pred_best[:, 1]))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0.7432498382630668


## Полезные ссылочки

[Примеры про LGB](https://nitin9809.medium.com/lightgbm-binary-classification-multi-class-classification-regression-using-python-4f22032b36a2)

[Важные отличия разных пакетов бустинга](https://medium.com/riskified-technology/xgboost-lightgbm-or-catboost-which-boosting-algorithm-should-i-use-e7fda7bb36bc)

## LightGBM боевого образца

In [ ]:
def target_feat_matrix(data):
    y = data['ARRIVAL_DELAY']
    X = data.drop('ARRIVAL_DELAY', axis=1)
    return X, y

In [ ]:
cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for col in cols:
    data[col] = data[col].astype('category').cat.codes + 1

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.1, shuffle=True)


In [ ]:
X_train, y_train = target_feat_matrix(data_train)
X_test, y_test = target_feat_matrix(data_test)

In [ ]:
params = {
    'objective': 'binary', 
    'learning_rate': 0.5,
    'n_estimators': 15,
    'max_depth': 4,
    'early_stopping_round': 5,
    'metric': 'auc',
    'feature_fraction': 0.7
    
}

In [ ]:
for i in range(3):
    data_train, data_valid = train_test_split(data_train, test_size=0.2, shuffle=True)
    X_valid, y_valid = target_feat_matrix(data_valid)
    model = lgb.train(params,
                  train_set=lgb.Dataset(X_train, label=y_train),
                  valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
                  categorical_feature=cols)
    model.save_model('model' + str(i) + '.txt')

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True)

In [ ]:
i = 0 
for train_index, valid_index in kf.split(data_train):
    train_data, valid_data = data_train.iloc[train_index], data_train.iloc[valid_index]
    X_train, y_train = target_feat_matrix(train_data)
    X_valid, y_valid = target_feat_matrix(valid_data)
    model = lgb.train(params,
                  train_set=lgb.Dataset(X_train, label=y_train),
                  valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
                  categorical_feature=cols)
    model.save_model('model' + str(i) + '.txt')
    i += 1

In [ ]:
prediction = pd.DataFrame()
for i in range(3):
    bst = lgb.Booster(model_file='model' + str(i) + '.txt')
    y_pred = bst.predict(X_test)
    prediction[f'model{i}'] = y_pred

In [ ]:
prediction['prediction'] = prediction.mean(axis=1)

In [ ]:
prediction